In [2]:
import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
import mhcflurry.data
import mhcflurry.imputation
import fancyimpute, locale

import sklearn.metrics
import sklearn.cross_validation

In [13]:
models = pandas.read_csv("../data/validation_models.csv", converters={'layer_sizes': eval})
models["layer_size"] = [x[0] for x in models.layer_sizes]
models

,activation,dropout_probability,embedding_output_dim,impute,layer_sizes,layer_size
0,tanh,0.0,32,False,[64],64
1,tanh,0.0,8,False,[4],4
2,tanh,0.5,32,False,[64],64
3,tanh,0.5,8,False,[4],4
4,tanh,0.0,32,True,[64],64
5,tanh,0.0,8,True,[4],4
6,tanh,0.5,32,True,[64],64
7,tanh,0.5,8,True,[4],4


In [20]:
df = pandas.read_csv("../data/validation_scores.csv", index_col="allele")
df

,mhcflurry 0_auc,mhcflurry 0_f1,mhcflurry 0_tau,mhcflurry 1_auc,mhcflurry 1_f1,mhcflurry 1_tau,mhcflurry 2_auc,mhcflurry 2_f1,mhcflurry 2_tau,mhcflurry 3_auc,...,netmhc_f1,netmhc_tau,netmhcpan_auc,netmhcpan_f1,netmhcpan_tau,smmpmbec_cpp_auc,smmpmbec_cpp_f1,smmpmbec_cpp_tau,test_size,train_size
allele,,,,,,,,,,,,,,,,,,,,,
HLA-A0201,0.899123,0.837826,0.554988,0.926853,0.870671,0.623314,0.925713,0.881167,0.622557,0.913962,...,0.884336,0.635498,0.930479,0.880963,0.637338,0.927358,0.885121,0.626224,2126,9565
HLA-A0301,0.891561,0.834236,0.540025,0.922007,0.892147,0.595890,0.923343,0.879402,0.602523,0.916512,...,0.900621,0.629236,0.927287,0.885106,0.611240,0.933966,0.897275,0.610891,811,6141
HLA-A0201,0.899123,0.837826,0.554988,0.926853,0.870671,0.623314,0.925713,0.881167,0.622557,0.913962,...,0.884336,0.635498,0.930479,0.880963,0.637338,0.927358,0.885121,0.626224,2126,9565
HLA-A0301,0.891561,0.834236,0.540025,0.922007,0.892147,0.595890,0.923343,0.879402,0.602523,0.916512,...,0.900621,0.629236,0.927287,0.885106,0.611240,0.933966,0.897275,0.610891,811,6141
HLA-A2601,0.928084,0.580000,0.432299,0.929260,0.593103,0.440722,0.931760,0.574899,0.433437,0.923544,...,0.541176,0.436646,0.942279,0.640000,0.451666,0.927339,0.603053,0.422900,1333,2894
HLA-B0801,0.942694,0.762931,0.600817,0.949517,0.773504,0.620342,0.948274,0.778523,0.616491,0.940561,...,0.776053,0.629519,0.942822,0.701671,0.606762,0.940555,0.760000,0.618953,940,2267
HLA-B5101,0.933050,0.489796,0.369192,0.939082,0.391304,0.369359,0.948141,0.404762,0.382398,0.945669,...,0.428571,0.365307,0.948695,0.610169,0.406973,0.927621,0.395604,0.344534,854,1734
HLA-B5701,0.914240,0.750000,0.524933,0.916777,0.740299,0.535227,0.922818,0.670886,0.545377,0.914248,...,0.741840,0.484975,0.883077,0.642623,0.460133,0.929923,0.709091,0.569615,815,1857
HLA-B0702,0.904985,0.792711,0.564505,0.915801,0.866525,0.594497,0.909308,0.835526,0.587247,0.898044,...,0.869383,0.606014,0.913527,0.855011,0.604720,0.897928,0.857719,0.563960,813,3412


In [29]:
threshold = 1000
big_model = 6
small_model = 5
extra_cols = collections.defaultdict(list)
for (allele, row) in df.iterrows():
    for metric in ["auc", "f1", "tau"]:
        model = big_model if row.train_size >= threshold else small_model
        extra_cols["mhcflurry standard_%s" % metric].append(row["mhcflurry %s_%s" % (model, metric)])
        
for (col, values) in extra_cols.items():
    df[col] = values

df

,mhcflurry 0_auc,mhcflurry 0_f1,mhcflurry 0_tau,mhcflurry 1_auc,mhcflurry 1_f1,mhcflurry 1_tau,mhcflurry 2_auc,mhcflurry 2_f1,mhcflurry 2_tau,mhcflurry 3_auc,...,netmhcpan_f1,netmhcpan_tau,smmpmbec_cpp_auc,smmpmbec_cpp_f1,smmpmbec_cpp_tau,test_size,train_size,mhcflurry standard_tau,mhcflurry standard_f1,mhcflurry standard_auc
allele,,,,,,,,,,,,,,,,,,,,,
HLA-A0201,0.899123,0.837826,0.554988,0.926853,0.870671,0.623314,0.925713,0.881167,0.622557,0.913962,...,0.880963,0.637338,0.927358,0.885121,0.626224,2126,9565,0.622718,0.874891,0.925388
HLA-A0301,0.891561,0.834236,0.540025,0.922007,0.892147,0.595890,0.923343,0.879402,0.602523,0.916512,...,0.885106,0.611240,0.933966,0.897275,0.610891,811,6141,0.601211,0.879144,0.922855
HLA-A0201,0.899123,0.837826,0.554988,0.926853,0.870671,0.623314,0.925713,0.881167,0.622557,0.913962,...,0.880963,0.637338,0.927358,0.885121,0.626224,2126,9565,0.622718,0.874891,0.925388
HLA-A0301,0.891561,0.834236,0.540025,0.922007,0.892147,0.595890,0.923343,0.879402,0.602523,0.916512,...,0.885106,0.611240,0.933966,0.897275,0.610891,811,6141,0.601211,0.879144,0.922855
HLA-A2601,0.928084,0.580000,0.432299,0.929260,0.593103,0.440722,0.931760,0.574899,0.433437,0.923544,...,0.640000,0.451666,0.927339,0.603053,0.422900,1333,2894,0.432822,0.553719,0.931268
HLA-B0801,0.942694,0.762931,0.600817,0.949517,0.773504,0.620342,0.948274,0.778523,0.616491,0.940561,...,0.701671,0.606762,0.940555,0.760000,0.618953,940,2267,0.616177,0.779736,0.949331
HLA-B5101,0.933050,0.489796,0.369192,0.939082,0.391304,0.369359,0.948141,0.404762,0.382398,0.945669,...,0.610169,0.406973,0.927621,0.395604,0.344534,854,1734,0.379767,0.418605,0.946123
HLA-B5701,0.914240,0.750000,0.524933,0.916777,0.740299,0.535227,0.922818,0.670886,0.545377,0.914248,...,0.642623,0.460133,0.929923,0.709091,0.569615,815,1857,0.548574,0.701863,0.923588
HLA-B0702,0.904985,0.792711,0.564505,0.915801,0.866525,0.594497,0.909308,0.835526,0.587247,0.898044,...,0.855011,0.604720,0.897928,0.857719,0.563960,813,3412,0.587399,0.835165,0.909692


In [51]:
df.mean().sort(inplace=False)

mhcflurry 4_tau              0.447560
mhcflurry 0_tau              0.449200
mhcflurry 3_tau              0.449642
mhcflurry 7_tau              0.462858
mhcflurry 5_tau              0.468839
mhcflurry 1_tau              0.471471
smmpmbec_cpp_tau             0.471492
mhcflurry 2_tau              0.472481
mhcflurry standard_tau       0.472755
mhcflurry 6_tau              0.474900
netmhc_tau                   0.483207
netmhcpan_tau                0.491046
mhcflurry 3_f1               0.533349
mhcflurry 7_f1               0.543681
mhcflurry 2_f1               0.630669
mhcflurry standard_f1        0.631175
mhcflurry 6_f1               0.635060
mhcflurry 5_f1               0.636095
mhcflurry 0_f1               0.636355
smmpmbec_cpp_f1              0.637798
mhcflurry 1_f1               0.644236
mhcflurry 4_f1               0.649554
netmhc_f1                    0.659745
netmhcpan_f1                 0.670119
mhcflurry 3_auc              0.881133
mhcflurry 0_auc              0.888551
mhcflurry 4_

In [50]:
df[df.train_size > 1000].mean().sort(inplace=False)

mhcflurry 4_tau              0.496057
mhcflurry 0_tau              0.502704
smmpmbec_cpp_tau             0.516050
mhcflurry 3_tau              0.522841
mhcflurry 7_tau              0.525517
mhcflurry 5_tau              0.528831
mhcflurry 1_tau              0.532539
mhcflurry 2_tau              0.534358
mhcflurry standard_tau       0.534703
mhcflurry 6_tau              0.534703
netmhcpan_tau                0.534936
netmhc_tau                   0.536833
mhcflurry 3_f1               0.603511
mhcflurry 7_f1               0.609657
smmpmbec_cpp_f1              0.707976
mhcflurry 0_f1               0.709276
mhcflurry standard_f1        0.710507
mhcflurry 6_f1               0.710507
mhcflurry 2_f1               0.711558
mhcflurry 4_f1               0.716505
mhcflurry 5_f1               0.717820
netmhcpan_f1                 0.720971
mhcflurry 1_f1               0.722733
netmhc_f1                    0.724166
mhcflurry 4_auc              0.883488
mhcflurry 0_auc              0.883998
smmpmbec_cpp

In [47]:
df[df.train_size < 200]

,mhcflurry 0_auc,mhcflurry 0_f1,mhcflurry 0_tau,mhcflurry 1_auc,mhcflurry 1_f1,mhcflurry 1_tau,mhcflurry 2_auc,mhcflurry 2_f1,mhcflurry 2_tau,mhcflurry 3_auc,...,netmhcpan_f1,netmhcpan_tau,smmpmbec_cpp_auc,smmpmbec_cpp_f1,smmpmbec_cpp_tau,test_size,train_size,mhcflurry standard_tau,mhcflurry standard_f1,mhcflurry standard_auc
allele,,,,,,,,,,,,,,,,,,,,,
HLA-B3801,0.901216,0,0.500359,0.909119,0,0.515732,0.921952,0,0.524527,0.725768,...,0,0.652173,0.949206,0,0.547438,351,136,0.523753,0,0.9359


In [49]:
df[df.train_size > 1000].mean().sort(inplace=False)

mhcflurry 4_tau              0.496057
mhcflurry 0_tau              0.502704
smmpmbec_cpp_tau             0.516050
mhcflurry 3_tau              0.522841
mhcflurry 7_tau              0.525517
mhcflurry 5_tau              0.528831
mhcflurry 1_tau              0.532539
mhcflurry 2_tau              0.534358
mhcflurry standard_tau       0.534703
mhcflurry 6_tau              0.534703
netmhcpan_tau                0.534936
netmhc_tau                   0.536833
mhcflurry 3_f1               0.603511
mhcflurry 7_f1               0.609657
smmpmbec_cpp_f1              0.707976
mhcflurry 0_f1               0.709276
mhcflurry standard_f1        0.710507
mhcflurry 6_f1               0.710507
mhcflurry 2_f1               0.711558
mhcflurry 4_f1               0.716505
mhcflurry 5_f1               0.717820
netmhcpan_f1                 0.720971
mhcflurry 1_f1               0.722733
netmhc_f1                    0.724166
mhcflurry 4_auc              0.883488
mhcflurry 0_auc              0.883998
smmpmbec_cpp

In [35]:
(df["mhcflurry standard_auc"] > df["netmhc_auc"])

allele
HLA-A0201    False
HLA-A0301    False
HLA-A0201    False
HLA-A0301    False
HLA-A2601     True
HLA-B0801     True
HLA-B5101     True
HLA-B5701     True
HLA-B0702    False
HLA-A3101    False
HLA-A1101    False
HLA-A0101    False
HLA-A0206    False
HLA-A6802    False
HLA-A3001     True
HLA-A0203    False
HLA-B3901    False
HLA-B1501     True
HLA-B1517     True
HLA-A2402     True
H-2-DB       False
H-2-KB        True
HLA-B3501    False
HLA-A6801    False
HLA-B0802    False
HLA-B1801    False
HLA-B5301     True
HLA-A3301     True
HLA-A6901    False
HLA-B1509    False
HLA-A3201     True
HLA-B5801     True
HLA-B2703    False
HLA-A3002    False
HLA-A2501    False
HLA-A2602    False
HLA-B4402    False
HLA-B4001    False
HLA-A2301    False
Mamu-A02     False
HLA-A8001    False
HLA-B4403    False
HLA-B4601    False
HLA-B3801     True
HLA-B2705    False
HLA-A2603    False
Mamu-A01      True
HLA-B0803    False
H-2-KD       False
HLA-B1503    False
HLA-A0202     True
HLA-A2902     True
HLA-B

In [38]:
seaborn.pointplot(data=df[[x for x in df.columns if 'auc' in x or x == "train_size"]].stack().reset_index(),
                  x=""

SyntaxError: invalid syntax (<ipython-input-38-6d5b57bb2869>, line 2)

In [30]:
df.mean()[[x for x in df.columns if 'auc' in x]]

mhcflurry 0_auc           0.888551
mhcflurry 1_auc           0.901088
mhcflurry 2_auc           0.903755
mhcflurry 3_auc           0.881133
mhcflurry 4_auc           0.890536
mhcflurry 5_auc           0.900119
mhcflurry 6_auc           0.903171
mhcflurry 7_auc           0.898431
netmhc_auc                0.909872
netmhcpan_auc             0.913077
smmpmbec_cpp_auc          0.895268
mhcflurry standard_auc    0.900856
dtype: float64

In [21]:
df.mean(axis=0)

mhcflurry 0_auc        0.888551
mhcflurry 0_f1         0.636355
mhcflurry 0_tau        0.449200
mhcflurry 1_auc        0.901088
mhcflurry 1_f1         0.644236
mhcflurry 1_tau        0.471471
mhcflurry 2_auc        0.903755
mhcflurry 2_f1         0.630669
mhcflurry 2_tau        0.472481
mhcflurry 3_auc        0.881133
mhcflurry 3_f1         0.533349
mhcflurry 3_tau        0.449642
mhcflurry 4_auc        0.890536
mhcflurry 4_f1         0.649554
mhcflurry 4_tau        0.447560
mhcflurry 5_auc        0.900119
mhcflurry 5_f1         0.636095
mhcflurry 5_tau        0.468839
mhcflurry 6_auc        0.903171
mhcflurry 6_f1         0.635060
mhcflurry 6_tau        0.474900
mhcflurry 7_auc        0.898431
mhcflurry 7_f1         0.543681
mhcflurry 7_tau        0.462858
netmhc_auc             0.909872
netmhc_f1              0.659745
netmhc_tau             0.483207
netmhcpan_auc          0.913077
netmhcpan_f1           0.670119
netmhcpan_tau          0.491046
smmpmbec_cpp_auc       0.895268
smmpmbec